In [ ]:
import os

# Path to your service account key file
key_path = r"C:\Users\hp\Downloads\epileptic-seizures-f935d9557a75.json"

# Set the environment variable
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = key_path

# Print the value of the environment variable
print(os.environ.get("GOOGLE_APPLICATION_CREDENTIALS"))


In [ ]:
from google.cloud import storage

import gcsfs
import mne
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import os


In [ ]:

input_bucket_name = 'chbmit-1.0.0.physionet.org'
output_bucket_name = 'epileptic-seizures-img'

def save_spectrogram_as_image(data, sfreq, channel_name, file_name, patient_id, edf_folder_name, channel_number, output_base_path, nperseg=256, noverlap=128):
    frequencies, times, Sxx = signal.spectrogram(data, fs=sfreq, nperseg=nperseg, noverlap=noverlap)
    plt.figure(figsize=(10, 5))
    plt.pcolormesh(times, frequencies, 10 * np.log10(Sxx), shading='gouraud')
    plt.ylabel('Frequency [Hz]')
    plt.xlabel('Time [sec]')
    plt.title(f'Spectrogram for {channel_name}')
    plt.colorbar(label='dB')
    plt.tight_layout()

    # Create the image name
    image_name = f"{patient_id}-{edf_folder_name}-{channel_name}.png"
    image_path = os.path.join(output_base_path, image_name)

    # Save the image locally
    plt.savefig(image_path)
    plt.close()

def read_edf_from_gcs(bucket_name, file_path):
# Create a client using the default credentials
    project_id = "epileptic-seizures"
    client = storage.Client(project=project_id, user_project=project_id)

# Access the bucket
    input_bucket = client.get_bucket(input_bucket_name)
    blobs = input_bucket.list_blobs()


    # Download the blob to a local file
    with tempfile.NamedTemporaryFile() as temp_file:
        blob.download_to_filename(temp_file.name)

        # Read the EDF data from the local file
        raw = mne.io.read_raw_edf(temp_file.name, preload=True)

    return raw

# Configuration
input_bucket_name = 'chbmit-1.0.0.physionet.org'
project_id = "epileptic-seizures"
client = storage.Client(project=project_id)
# Access the bucket
client._base_connection.user_project = "epileptic-seizures"
# Access the bucket
input_bucket = client.get_bucket(input_bucket_name, user_project=project_id)

blobs = input_bucket.list_blobs()
# Configuration
output_base_path = r"F:\Deep Learning\Spectrograms"

# Process data
for blob in blobs:
    if blob.name.endswith('.edf'):
        raw = read_edf_from_gcs(input_bucket_name, blob.name)
        person_id = os.path.splitext(os.path.basename(blob.name))[0]

        for channel_index in range(len(raw.ch_names)):
            if channel_index not in [14, 22]:  # Skip the duplicate channels (indices 14 and 22)
                channel_data = raw.get_data()[channel_index, :]
                sfreq = raw.info['sfreq']
                channel_name = raw.ch_names[channel_index]
                file_name = os.path.basename(blob.name)
                save_spectrogram_as_image(channel_data, sfreq, channel_name, file_name, person_id, person_id, channel_index, output_base_path)
